# Day 9: Disk Fragmenter

## Import libraries

In [2]:
import copy

## Import data

In [ ]:
# *** [IMPORT DATA] ***
# NOTE: In the given puzzle input:
# - A single string line that represents a disk map.
# - The disk map uses a dense format to represent the layout of files and free space on the disk.
# - The digits alternate between indicating the *length of a file* and the *length of free space*.
# - E.g. A disk map = '12345' represents: 1-block file; 2 blocks of free space; 3-block file; 4-blocks of free space; 5-block file.
# =====================================================================================================================
# ! Open the file for reading mode (= default mode if the mode is not specified)
file = open("../data/24_day-9_input.txt", "r") 

# Read all the data in the file
file_data = file.read().strip()

print(file_data)
# ====================================================================================================================

## Helper functions

In [4]:
def rearrange_disk_map(s):
    left = 0  # Start from the beginning of the string
    right = len(s) - 1  # Start from the end of the string

    while left < right:
        # If the current character is a '.', find the first number from the right
        if s[left] == '.':
            # Traverse from the right to find the first number
            while right > left and s[right] == '.':
                right -= 1
            # Swap the '.' with the number
            if right > left:
                s[left], s[right] = s[right], s[left]
                right -= 1
        left += 1

    # Return the rearranged string
    return s

In [ ]:
def swap_dot_with_digits(input_string): # INCORRECT - NEEDS TO BE FIXED
    # Convert the string into a list for easy manipulation
    input_list = list(input_string)

    # Left pointer for the dots
    left_pointer = 0
    while left_pointer < len(input_list) and input_list[left_pointer] != '.':
        left_pointer += 1

    # Continue swapping until no more valid swaps can be made
    while left_pointer < len(input_list):
        # Count the number of dots
        dot_start = left_pointer
        while left_pointer < len(input_list) and input_list[left_pointer] == '.':
            left_pointer += 1
        dot_length = left_pointer - dot_start

        # Find the first sequence of same digits from the right
        right_pointer = len(input_list) - 1
        while right_pointer > dot_start:
            digit_start = right_pointer
            while digit_start > dot_start and input_list[digit_start] == input_list[right_pointer]:
                digit_start -= 1
            digit_length = right_pointer - digit_start

            # Check if the digit sequence can fit into the dot sequence
            if dot_length >= digit_length:
                # Perform the swap
                for i in range(digit_length):
                    input_list[dot_start + i] = input_list[right_pointer - digit_length + 1 + i]
                for i in range(digit_length):
                    input_list[right_pointer - digit_length + 1 + i] = '.'
                break
            else:
                # Move to the next sequence of digits on the right
                right_pointer = digit_start

        # Move the left pointer to the next dot sequence
        while left_pointer < len(input_list) and input_list[left_pointer] != '.':
            left_pointer += 1

    # Convert the list back into a string
    return ''.join(input_list)

## Part 1

In [ ]:
# *** [PART 1] ***
# ! PROBLEM: The amphipod would like to move file blocks *one at a time* from the END of the disk to the LEFTMOST free space block (until there are no gaps remaining between file blocks - see examples on website).
# - Each file on disk also has an ID number based on the *order* of the files as they appear BEFORE they are rearranged, starting with ID 0.
# - E.g. A disk map '12345' has 3 files: 1-block file (ID 0); 3-block file (ID 1); 5-block file (ID 2).
# - - 'x' ID digits are used to represent EACH block (where 'x' = file block number) & '.' = free space.
# - E.g. A disk map '12345' becomes '0..111....22222' after rearrangement.
# - TODO: Calculate the resulting filesystem checksum of the RE-ARRANGED & MOVED disk map: Add up the result of multiplying each of the blocks' position with the file ID number it contains.
# - E.g. '0099811188827773336446555566...' = '(0 * 0 = 0) + (1 * 0 = 0) + (2 * 9 = 18) + (3 * 9 = 27) + (4 * 8 = 32) + ... ' (Multiply array item idx by value)
# ====================================================================================================================
# ! Create a deep (independent) copy of the data, such that changes made to the copy do not affect the original data to still test/re-run Part 1/2 with the correct INITIAL (and not modified) data
# - NOTE: Not using a deep copy will modify the original data after running Part 1/2, therefore no correct output will be calculated anymore.
disk_map = copy.deepcopy(file_data)
arrTransformedDiskMap = []
rearrangedDiskMap = ''
blockCounter = 0
checkSum = 0

for i in range(len(disk_map)):
    if i % 2 == 0: # If the index is even (i.e., we're looking at a block)
        for j in range(int(disk_map[i])): #'x' ID digits are used to represent EACH block (where 'x' = file block number)
            # NOTE: DO NOT use 'transformedDiskMap +=' because if 'blockCounter' number > 1 digit, then when transform string to a list, it will break up all numbers in the string into singluar digits (E.g. '10' => '1','0'), therefore append each number AS A WHOLE into an array list
            arrTransformedDiskMap.append(str(blockCounter))
        blockCounter += 1
    elif i % 2 != 0: # If the index is odd (i.e., we're looking at a free space)
        for j in range(int(disk_map[i])):
            arrTransformedDiskMap.append('.') # Append '.' as many times as the CURRENT free space number

""" Re-arrange transformed disk map """
# Move file blocks (ID numbers) from the end of the disk to the leftmost free space block (until there are no gaps remaining between file blocks)
rearrangedDiskMap = rearrange_disk_map(arrTransformedDiskMap)
# print(rearrangedDiskMap)

for i in range(len(rearrangedDiskMap)):
    if rearrangedDiskMap[i] != '.':
        checkSum += (i * int(rearrangedDiskMap[i]))

print("Filesystem checksum (Part 1):", checkSum)
# ====================================================================================================================

## Part 2

In [ ]:
# *** [PART 2] ***
# ! PROBLEM: Upon completion, two things immediately become clear. First, the disk definitely has a lot more contiguous free space, just like the amphipod hoped. Second, the computer is running much more slowly! Maybe introducing all of that file system fragmentation was a bad idea?
# - TODO: Rather than moving individual blocks, compact disk files by moving WHOLE files instead.
# - Attempt to move *whole files* to the *leftmost span* of free space blocks that could fit the file.
# - Attempt to move each file *exactly once* in order of *decreasing file ID number*, starting with the file with the *highest file ID number*.
# - If there is no span of free space to the left of a file that is large enough to fit the file, the file does not move.
#====================================================================================================================
# how can i check the same seq of numbers in an array list?




# # Test the function
# input_string = '00...111...2...333.44.5555.6666.777.888899'
# expected_output = '00992111777.44.333....5555.6666.....8888..'

# output = swap_dot_with_digits(input_string)
# print("Output:", output)
# print("Matches expected output:", output == expected_output)

